In [19]:
!ngrok authtoken 2i7lMytmZ8I8f7IoCqZBnQCD4CC_81EP58mqr8ntWDpEcMpFs

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [20]:
!pip install streamlit pyngrok tensorflow opencv-python


In [21]:
%%writefile streamlit_app.py
import streamlit as st
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2

# Các hằng số
IMAGE_W = 224
IMAGE_H = 224

# Định nghĩa mô hình
def get_model():
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
    for layer in model_vgg16_conv.layers:
        layer.trainable = False
    input = Input(shape=(IMAGE_W, IMAGE_H, 3), name='image_input')
    output_vgg16_conv = model_vgg16_conv(input)
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dropout(0.5)(x)
    x = Dense(5, activation='sigmoid', name='predictions')(x)
    my_model = Model(inputs=input, outputs=x)
    my_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['BinaryAccuracy'])
    return my_model

model = get_model()

# Giao diện Streamlit
st.title('Phân loại hình ảnh đa nhãn')

uploaded_file = st.file_uploader("Chọn một hình ảnh...", type="jpg")
if uploaded_file is not None:
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, 1)
    img_resized = cv2.resize(img, (IMAGE_W, IMAGE_H))
    st.image(img, caption='Hình ảnh đã tải lên', use_column_width=True)

    img_array = image.img_to_array(img_resized)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)

    prediction = model.predict(img_array)

    st.write('Dự đoán:', prediction)


Overwriting streamlit_app.py


In [22]:
# Khởi chạy Streamlit trong nền
!nohup streamlit run streamlit_app.py &


nohup: appending output to 'nohup.out'


In [23]:
from pyngrok import ngrok

# Tạo đường hầm tới cổng 8501 của Streamlit
public_url = ngrok.connect(addr="8501")
print(f"URL truy cập công khai: {public_url}")


URL truy cập công khai: NgrokTunnel: "https://ae83-34-83-101-150.ngrok-free.app" -> "http://localhost:8501"
